<a href="https://colab.research.google.com/github/rorschach02/Human-Activity-Recognition/blob/main/Human_Acitivity_Recognition_LSTMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Importing Libraries

import pandas as pd
import numpy as np

# Import Keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout
from keras.layers import BatchNormalization
from keras.regularizers import L1L2


In [4]:
import matplotlib.pyplot as plt
import seaborn as sns


In [5]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

def confusion_matrix(Y_true, Y_pred):
    
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])



Loading Data:

In [6]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
    ]


In [20]:
# function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'drive/MyDrive/UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).to_numpy()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))


In [21]:
def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'drive/MyDrive/UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).to_numpy()


In [22]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test


In [23]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)


In [24]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32


In [25]:
#function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))


In [26]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()


In [27]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))


128
9
7352


1. Defining the Architecture of 1-Layer of LSTM :

In [28]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                5376      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 6)                 198       
                                                                 
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


In [30]:
# Training the model
model.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test),epochs=epochs)


Epoch 1/30
460/460 [==============================] - 26s 52ms/step - loss: 1.2052 - accuracy: 0.4944 - val_loss: 0.9226 - val_accuracy: 0.5945
Epoch 2/30
460/460 [==============================] - 24s 52ms/step - loss: 0.8041 - accuracy: 0.6500 - val_loss: 0.7612 - val_accuracy: 0.6841
Epoch 3/30
460/460 [==============================] - 24s 52ms/step - loss: 0.6805 - accuracy: 0.7203 - val_loss: 0.6757 - val_accuracy: 0.7418
Epoch 4/30
460/460 [==============================] - 24s 52ms/step - loss: 0.5312 - accuracy: 0.8111 - val_loss: 0.7307 - val_accuracy: 0.7608
Epoch 5/30
460/460 [==============================] - 24s 53ms/step - loss: 0.3963 - accuracy: 0.8723 - val_loss: 0.4535 - val_accuracy: 0.8585
Epoch 6/30
460/460 [==============================] - 24s 51ms/step - loss: 0.3088 - accuracy: 0.9101 - val_loss: 0.4747 - val_accuracy: 0.8504
Epoch 7/30
460/460 [==============================] - 24s 52ms/step - loss: 0.2765 - accuracy: 0.9189 - val_loss: 0.4399 - val_accuracy:

In [32]:
# Confusion Matrix
confusion_matrix(Y_test, model.predict(X_test))


Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,510,0,27,0,0,0
SITTING,3,408,77,0,1,2
STANDING,0,83,447,1,0,1
WALKING,0,0,1,446,24,25
WALKING_DOWNSTAIRS,0,0,0,0,420,0
WALKING_UPSTAIRS,0,0,0,1,33,437


In [33]:
score = model.evaluate(X_test, Y_test)

print("\n   cat_crossentropy  ||   accuracy ")
print("  ____________________________________")
print(score)


93/93 [==============================] - 1s 14ms/step - loss: 0.6416 - accuracy: 0.9053

   cat_crossentropy  ||   accuracy 
  ____________________________________
[0.6415585875511169, 0.9053274393081665]


2. Defining the Architecture of 2-Layer of LSTM with more hyperparameter tunning



In [34]:
# Initializing parameters
n_epochs = 30
n_batch = 16
n_classes = _count_classes(Y_train)

# Bias regularizer value - we will use elasticnet
reg = L1L2(0.01, 0.01)


In [35]:
# Model execution
model = Sequential()
model.add(LSTM(48, input_shape=(timesteps, input_dim), return_sequences=True,bias_regularizer=reg ))
model.add(BatchNormalization())
model.add(Dropout(0.50))
model.add(LSTM(32))
model.add(Dropout(0.50))
model.add(Dense(n_classes, activation='sigmoid'))
print("Model Summary: ")
model.summary()


Model Summary: 
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 128, 48)           11136     
                                                                 
 batch_normalization (BatchN  (None, 128, 48)          192       
 ormalization)                                                   
                                                                 
 dropout_1 (Dropout)         (None, 128, 48)           0         
                                                                 
 lstm_2 (LSTM)               (None, 32)                10368     
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 6)                 198       
                                      

In [36]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [37]:
# Training the model
model.fit(X_train, Y_train, batch_size=n_batch, validation_data=(X_test, Y_test), epochs=n_epochs)


Epoch 1/30
460/460 [==============================] - 60s 122ms/step - loss: 1.5615 - accuracy: 0.6718 - val_loss: 1.0155 - val_accuracy: 0.8164
Epoch 2/30
460/460 [==============================] - 55s 120ms/step - loss: 0.7513 - accuracy: 0.8659 - val_loss: 0.6000 - val_accuracy: 0.8646
Epoch 3/30
460/460 [==============================] - 55s 120ms/step - loss: 0.4851 - accuracy: 0.8908 - val_loss: 0.3591 - val_accuracy: 0.9043
Epoch 4/30
460/460 [==============================] - 55s 120ms/step - loss: 0.2675 - accuracy: 0.9275 - val_loss: 0.5049 - val_accuracy: 0.8415
Epoch 5/30
460/460 [==============================] - 55s 120ms/step - loss: 0.2199 - accuracy: 0.9252 - val_loss: 0.3338 - val_accuracy: 0.8890
Epoch 6/30
460/460 [==============================] - 55s 120ms/step - loss: 0.3425 - accuracy: 0.8932 - val_loss: 0.3935 - val_accuracy: 0.8700
Epoch 7/30
460/460 [==============================] - 55s 120ms/step - loss: 0.2532 - accuracy: 0.9202 - val_loss: 0.2999 - val_ac

In [38]:
confusion_matrix(Y_test, model.predict(X_test))

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,537,0,0,0,0,0
SITTING,4,373,112,0,0,2
STANDING,0,59,473,0,0,0
WALKING,0,0,0,494,0,2
WALKING_DOWNSTAIRS,0,0,0,18,366,36
WALKING_UPSTAIRS,1,5,0,3,0,462


In [39]:
score = model.evaluate(X_test, Y_test)

print("\n   cat_crossentropy  ||   accuracy ")
print("  ____________________________________")
print(score)


93/93 [==============================] - 3s 29ms/step - loss: 0.3114 - accuracy: 0.9179

   cat_crossentropy  ||   accuracy 
  ____________________________________
[0.3113946318626404, 0.9178826212882996]
